In [ ]:
!pip install tensorflow-addons
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.metrics import Precision, Recall
from tensorflow_addons.metrics import F1Score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support
import ast


In [ ]:
lyrics_df = pd.read_csv("/content/lyrics.csv")
df = lyrics_df.drop(['video_id', 'recording_ids','selected_mbid','genres_mb','genres_ab','genres_self','genres_genius',], axis=1)
df['lyrics'] = df['lyrics'].str.replace(r'[\s\n\t]+', ' ', regex=True).str.lower()



In [ ]:
df

,title,artist,genius_id,mood,lyrics,final_genres
0,(Drawing) Rings Around the World,Super Furry Animals,820634,['relaxed'],you expose the film in me we're drawing rings ...,"['electronic', 'rock']"
1,The Bitter End,Placebo,453094,['relaxed'],[verse 1] since we're feeling so anaesthetised...,['rock']
2,Stellar (acoustic),Incubus,337741,['sad'],[intro] (you are stellar) [verse 1] meet me in...,"['rock', 'funk&soul']"
3,Apply Some Pressure (radio edit),Maxïmo Park,332496,"['happy', 'aggressive']",[verse 1] you know that i would love to see yo...,['rock']
4,American English,Idlewild,1728125,[],songs when they're true are all dedicated to y...,['rock']
...,...,...,...,...,...,...
1467,Bad to the Bone,George Thorogood & the Destroyers,489000,"['party', 'happy']","[intro] [verse 1] on the day i was born, the n...","['rock', 'blues']"
1468,Every Day I Have the Blues,B.B. King,1784259,['relaxed'],"[b. b. king chorus] everyday, everyday i have ...",['blues']
1469,Ghetto Woman,B.B. King,1463301,"['relaxed', 'happy']","she's a ghetto woman, waitin for her man to co...","['blues', 'rock']"
1470,Blue and Lonesome,Little Walter,1450818,"['relaxed', 'sad']",[verse 1] i'm blue and lonesome as a man can b...,['blues']


In [ ]:
data=df

In [ ]:
df['lyrics']

0       you expose the film in me we're drawing rings ...
1       [verse 1] since we're feeling so anaesthetised...
2       [intro] (you are stellar) [verse 1] meet me in...
3       [verse 1] you know that i would love to see yo...
4       songs when they're true are all dedicated to y...
                              ...                        
1467    [intro] [verse 1] on the day i was born, the n...
1468    [b. b. king chorus] everyday, everyday i have ...
1469    she's a ghetto woman, waitin for her man to co...
1470    [verse 1] i'm blue and lonesome as a man can b...
1471    you made me leave my happy home you took my lo...
Name: lyrics, Length: 1472, dtype: object

In [ ]:

data['final_genres'] = data['final_genres'].apply(ast.literal_eval)

# Initialize the MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Transform 'final_genres' to a format suitable for model training
genres_encoded = mlb.fit_transform(data['final_genres'])

# Create a DataFrame for the encoded genres
genres_encoded_df = pd.DataFrame(genres_encoded, columns=mlb.classes_)

# Display the first few rows of the transformed genres
genres_encoded_df.head()


,blues,classical,electronic,folk&country,funk&soul,hiphop,jazz,latin,pop,rock
0,0,0,1,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,1,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,1


In [ ]:


# Initialize TF-IDF Vectorizer
tfidf = TfidfVectorizer(max_features=5000) # Limiting to 5000 features for performance

# Fit and transform the 'lyrics' column
tfidf_features = tfidf.fit_transform(data['lyrics'])

# Convert to a DataFrame
tfidf_features_df = pd.DataFrame(tfidf_features.toarray(), columns=tfidf.get_feature_names_out())

# Display the first few rows of the TF-IDF features
tfidf_features_df.head()


,10,100,11,12,15,16,17,1984,20,21,...,yummy,yuna,zal,zayn,zephyr,zero,zip,ziploc,zone,zoom
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.058218,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [ ]:


# Combine the TF-IDF features with the encoded genres
X = tfidf_features_df
y = genres_encoded_df

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape


((1177, 5000), (295, 5000), (1177, 10), (295, 10))

TF-IDF with RandomForest

In [ ]:

rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
classification_report_str = classification_report(y_test, y_pred, target_names=mlb.classes_)

accuracy, classification_report_str


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.3288135593220339,
 '              precision    recall  f1-score   support\n\n       blues       0.00      0.00      0.00        25\n   classical       0.00      0.00      0.00         0\n  electronic       1.00      0.06      0.12        16\nfolk&country       1.00      0.05      0.09        61\n   funk&soul       0.00      0.00      0.00        24\n      hiphop       0.98      0.71      0.82        59\n        jazz       0.00      0.00      0.00         6\n       latin       0.00      0.00      0.00         2\n         pop       1.00      0.07      0.12       105\n        rock       0.65      0.88      0.75       133\n\n   micro avg       0.72      0.39      0.51       431\n   macro avg       0.46      0.18      0.19       431\nweighted avg       0.76      0.39      0.39       431\n samples avg       0.57      0.44      0.48       431\n')

In [ ]:
accuracy

0.3288135593220339

In [ ]:
classification_report_str

'              precision    recall  f1-score   support\n\n       blues       0.00      0.00      0.00        25\n   classical       0.00      0.00      0.00         0\n  electronic       1.00      0.06      0.12        16\nfolk&country       1.00      0.05      0.09        61\n   funk&soul       0.00      0.00      0.00        24\n      hiphop       0.98      0.71      0.82        59\n        jazz       0.00      0.00      0.00         6\n       latin       0.00      0.00      0.00         2\n         pop       1.00      0.07      0.12       105\n        rock       0.65      0.88      0.75       133\n\n   micro avg       0.72      0.39      0.51       431\n   macro avg       0.46      0.18      0.19       431\nweighted avg       0.76      0.39      0.39       431\n samples avg       0.57      0.44      0.48       431\n'

In [ ]:
#               precision    recall  f1-score   support

#        blues       0.00      0.00      0.00        25
#    classical       0.00      0.00      0.00         0
#   electronic       1.00      0.06      0.12        16
# folk&country       1.00      0.05      0.09        61
#    funk&soul       0.00      0.00      0.00        24
#       hiphop       0.98      0.71      0.82        59
#         jazz       0.00      0.00      0.00         6
#        latin       0.00      0.00      0.00         2
#          pop       1.00      0.07      0.12       105
#         rock       0.65      0.88      0.75       133

#    micro avg       0.72      0.39      0.51       431
#    macro avg       0.46      0.18      0.19       431
# weighted avg       0.76      0.39      0.39       431
#  samples avg       0.57      0.44      0.48       431

TF-IDF with neural network

In [ ]:
f1_score = F1Score(num_classes=10, threshold=0.5, average='micro') # Adjust as needed

# Define the model
model = Sequential([
    Dense(1024, activation='relu', input_shape=(5000,)),  # Adjust the input shape
    Dropout(0.1),
    Dense(10, activation='sigmoid')  # 10 output units for 10 genres
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', Precision(), Recall(), f1_score])

model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 1024)              5121024   
                                                                 
 dropout_5 (Dropout)         (None, 1024)              0         
                                                                 
 dense_11 (Dense)            (None, 10)                10250     
                                                                 
Total params: 5131274 (19.57 MB)
Trainable params: 5131274 (19.57 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Convert data to NumPy arrays if not already
X_train_np = X_train.values
y_train_np = y_train.values
X_test_np = X_test.values
y_test_np = y_test.values

# Train the model
history = model.fit(X_train_np, y_train_np, epochs=10, batch_size=32, validation_data=(X_test_np, y_test_np))


Epoch 1/10
37/37 [==============================] - 8s 127ms/step - loss: 0.4563 - accuracy: 0.2099 - precision_2: 0.4072 - recall_2: 0.0918 - f1_score: 0.1498 - val_loss: 0.3345 - val_accuracy: 0.2000 - val_precision_2: 0.7034 - val_recall_2: 0.1926 - val_f1_score: 0.3024
Epoch 2/10
37/37 [==============================] - 4s 105ms/step - loss: 0.3046 - accuracy: 0.3110 - precision_2: 0.8193 - recall_2: 0.2317 - f1_score: 0.3612 - val_loss: 0.2974 - val_accuracy: 0.3288 - val_precision_2: 0.6528 - val_recall_2: 0.3271 - val_f1_score: 0.4359
Epoch 3/10
37/37 [==============================] - 6s 177ms/step - loss: 0.2526 - accuracy: 0.4002 - precision_2: 0.8021 - recall_2: 0.3978 - f1_score: 0.5318 - val_loss: 0.2695 - val_accuracy: 0.4136 - val_precision_2: 0.7512 - val_recall_2: 0.3573 - val_f1_score: 0.4843
Epoch 4/10
37/37 [==============================] - 5s 123ms/step - loss: 0.2045 - accuracy: 0.5038 - precision_2: 0.9226 - recall_2: 0.5604 - f1_score: 0.6973 - val_loss: 0.2594

In [ ]:
# Training

# loss: 9.9931e-04
# accuracy: 0.7596
# precision: 0.9994
# recall: 1.0000
# f1_score: 0.9997


# Validation:

# loss: 0.5035
# accuracy: 0.4169
# precision: 0.6403
# recall: 0.4501
# f1_score: 0.5286


N grams and Random forest

In [ ]:


# Reinitialize CountVectorizer for n-gram features (unigrams and bigrams)
ngram_vectorizer = CountVectorizer(max_features=5000, ngram_range=(1, 2))  # Adjust ngram_range as needed

# Apply n-gram vectorization to 'lyrics'
ngram_features = ngram_vectorizer.fit_transform(data['lyrics'])

# Convert to a DataFrame
ngram_features_df = pd.DataFrame(ngram_features.toarray(), columns=ngram_vectorizer.get_feature_names_out())

# Display the shape and first few rows of the n-gram features
ngram_features_df.shape, ngram_features_df.head()


((1472, 5000),
    10  12  20  24  40  50  aah  aah don  about  about it  ...  yourself and  \
 0   0   0   0   0   0   0    0        0      0         0  ...             0   
 1   0   0   0   0   0   0    0        0      0         0  ...             0   
 2   0   0   0   0   0   0    0        0      0         0  ...             0   
 3   0   0   0   0   0   0    0        0      3         0  ...             0   
 4   0   0   0   0   0   0    0        0     11         0  ...             0   
 
    youth  yoy  yoy yoy  yuh  yum  yummy  zero  zero zero  zone  
 0      0    0        0    0    0      0     0          0     0  
 1      0    0        0    0    0      0     0          0     1  
 2      0    0        0    0    0      0     0          0     0  
 3      0    0        0    0    0      0     0          0     0  
 4      2    0        0    0    0      0     0          0     0  
 
 [5 rows x 5000 columns])

In [ ]:


# Split the data into training and testing sets
X_train_ng, X_test_ng, y_train_ng, y_test_ng = train_test_split(ngram_features_df, genres_encoded_df, test_size=0.2, random_state=42)

# Initialize the Random Forest Classifier
rf_classifier_ng = RandomForestClassifier(random_state=42)

# Train the classifier
rf_classifier_ng.fit(X_train_ng, y_train_ng)

# Predict on the test set
y_pred_ng = rf_classifier_ng.predict(X_test_ng)

# Evaluate the model
accuracy_ng = accuracy_score(y_test_ng, y_pred_ng)
precision_ng, recall_ng, f1_score_ng, _ = precision_recall_fscore_support(y_test_ng, y_pred_ng, average='micro')

accuracy_ng, precision_ng, recall_ng, f1_score_ng


(0.3254237288135593,
 0.7142857142857143,
 0.39443155452436196,
 0.5082212257100149)

In [ ]:
# Test:

# accuracy- 0.32
# precision - 0.71
# recall - 0.39
# f1 score - 0.50

N gram with neural network

In [ ]:
model = Sequential([
    Dense(1024, activation='relu', input_shape=(5000,)),  # Adjust the input shape based on your feature size
    Dropout(0.1),
    Dense(10, activation='sigmoid')  # 10 output units for 10 genres
])


In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy',Precision(), Recall(), f1_score])


In [ ]:
history = model.fit(X_train_np, y_train_np,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)


Epoch 1/10
30/30 [==============================] - 4s 82ms/step - loss: 0.4849 - accuracy: 0.2125 - precision_3: 0.2500 - recall_3: 0.0387 - f1_score: 0.2124 - val_loss: 0.3581 - val_accuracy: 0.1610 - val_precision_3: 0.6697 - val_recall_3: 0.2080 - val_f1_score: 0.3174
Epoch 2/10
30/30 [==============================] - 2s 73ms/step - loss: 0.3162 - accuracy: 0.2614 - precision_3: 0.7613 - recall_3: 0.2699 - f1_score: 0.3985 - val_loss: 0.3249 - val_accuracy: 0.2585 - val_precision_3: 0.6731 - val_recall_3: 0.1994 - val_f1_score: 0.3077
Epoch 3/10
30/30 [==============================] - 2s 71ms/step - loss: 0.2675 - accuracy: 0.3826 - precision_3: 0.8033 - recall_3: 0.3574 - f1_score: 0.4947 - val_loss: 0.3003 - val_accuracy: 0.3051 - val_precision_3: 0.7459 - val_recall_3: 0.2593 - val_f1_score: 0.3848
Epoch 4/10
30/30 [==============================] - 3s 89ms/step - loss: 0.2214 - accuracy: 0.5058 - precision_3: 0.9092 - recall_3: 0.4894 - f1_score: 0.6363 - val_loss: 0.2859 - v

In [ ]:
X_test_np.shape

(295, 5000)

In [ ]:
y_test_np.shape

(295, 10)

In [ ]:
evaluation_metrics = model.evaluate(X_test_np, y_test_np)
print(f"Test Loss: {evaluation_metrics[0]}")
print(f"Test Accuracy: {evaluation_metrics[1]}")



10/10 [==============================] - 0s 24ms/step - loss: 0.2868 - accuracy: 0.4339 - precision_3: 0.6830 - recall_3: 0.4200 - f1_score: 0.5201
Test Loss: 0.28676560521125793
Test Accuracy: 0.43389829993247986


In [ ]:


y_pred = model.predict(X_test_np)
y_pred = (y_pred > 0.5)

report = classification_report(y_test_np, y_pred, target_names=mlb.classes_)
print(report)


10/10 [==============================] - 0s 21ms/step
              precision    recall  f1-score   support

       blues       1.00      0.04      0.08        25
   classical       0.00      0.00      0.00         0
  electronic       1.00      0.06      0.12        16
folk&country       0.63      0.39      0.48        61
   funk&soul       0.00      0.00      0.00        24
      hiphop       0.93      0.71      0.81        59
        jazz       0.00      0.00      0.00         6
       latin       0.00      0.00      0.00         2
         pop       0.57      0.31      0.40       105
        rock       0.68      0.60      0.64       133

   micro avg       0.68      0.42      0.52       431
   macro avg       0.48      0.21      0.25       431
weighted avg       0.66      0.42      0.48       431
 samples avg       0.51      0.45      0.46       431



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
